In [1]:
import pandas as pd
import yfinance as yf
import requests
from convert_date import *

In [17]:
s = requests.Session()

max_tickers = 30

# user-agent from https://useragentstring.com/
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}
res = s.get("https://api.nasdaq.com/api/quote/list-type/nasdaq100", headers=headers)

main_data = res.json()['data']['data']['rows']

if res.status_code == 200:

    main_data = res.json()['data']['data']['rows']
    df_index = range(len(main_data))
    df_final_index  = range(max_tickers)
    
    df_market_cap = pd.DataFrame(columns=['Symbol', 'Market Cap'], index=df_index)
    df_final = pd.DataFrame(columns=['Symbol', 'Company Name', 'Market Cap'], index=df_final_index)

    for i in df_index:
    
        md = main_data[i]
        df_market_cap.loc[i, 'Symbol'] = md['symbol']
        df_market_cap.loc[i, 'Market Cap'] = md['marketCap'].replace(',', '')
    
    df_market_cap['Market Cap'] = df_market_cap['Market Cap'].astype('Int64')
    df_market_cap = df_market_cap.sort_values(by='Market Cap', ascending=False)
    df_market_cap = df_market_cap.reset_index(drop=True)
    df_market_cap = df_market_cap[: max_tickers]

    df_final['Symbol'] = df_market_cap['Symbol']
    df_final['Market Cap'] = df_market_cap['Market Cap']

    for i in df_final_index:

        ticker = df_final.loc[i, 'Symbol']
        short_name = yf.Ticker(ticker).info['shortName']
        long_name = yf.Ticker(ticker).info['longName']
        company_name = short_name if len(short_name) <= len(long_name) else long_name
        df_final.loc[i, 'Company Name'] = company_name
        
    print(df_final)

   Symbol                    Company Name     Market Cap
0    AAPL                      Apple Inc.  3535119264280
1    MSFT           Microsoft Corporation  3370922292869
2    NVDA              NVIDIA Corporation  3179089716203
3    GOOG                   Alphabet Inc.  2308227240000
4   GOOGL                   Alphabet Inc.  2287095060000
5    AMZN                Amazon.com, Inc.  2023984965943
6    META            Meta Platforms, Inc.  1265400811864
7    TSLA                     Tesla, Inc.   791654164534
8    AVGO                   Broadcom Inc.   791642113011
9    ASML               ASML Holding N.V.   426964856932
10   COST    Costco Wholesale Corporation   373687091730
11    AMD    Advanced Micro Devices, Inc.   293538762112
12   NFLX                   Netflix, Inc.   279092928172
13   ADBE                      Adobe Inc.   247882770000
14    AZN                 Astrazeneca PLC   245674006286
15    PEP                   Pepsico, Inc.   228534975912
16   QCOM           QUALCOMM In

In [23]:
sp500_url = 'https://stockanalysis.com/list/sp-500-stocks/'
res_sp500 = s.get(sp500_url, headers=headers)
cols = ['Symbol', 'Company Name', 'Stock Price', 'Market Cap']
if res_sp500.status_code == 200:
    # Index 0 means take the first dataframe from the extracted list
    df_sp500 = pd.read_html(res_sp500.content)[0][cols]
    df_sp500['Market Cap'] = df_sp500['Market Cap'].apply(lambda x: x.replace(',', '').replace('B', '')).astype(float)
    df_sp500 = df_sp500.sort_values(by='Market Cap', ascending=False)
    df_sp500 = df_sp500.rename(columns={'Market Cap': 'Market Cap ($B)'})
    print(df_sp500)

    Symbol                  Company Name  Stock Price  Market Cap ($B)
0     AAPL                     Apple Inc       234.40          3594.31
1     MSFT         Microsoft Corporation       453.96          3373.97
2     NVDA            NVIDIA Corporation       128.44          3159.62
3    GOOGL                 Alphabet Inc.       186.53          2314.82
4     GOOG                 Alphabet Inc.       188.19          2314.04
..     ...                           ...          ...              ...
498    BWA               BorgWarner Inc.        33.18             7.56
499    IVZ                  Invesco Ltd.        16.22             7.30
500    FMC               FMC Corporation        58.06             7.25
501   ETSY                    Etsy, Inc.        61.57             7.20
502    AAL  American Airlines Group Inc.        10.63             6.97

[503 rows x 4 columns]


In [29]:
djia_url = 'https://stockanalysis.com/list/dow-jones-stocks/'
res_djia = s.get(djia_url, headers=headers)
cols = ['Symbol', 'Company Name', 'Stock Price', 'Market Cap']
if res_djia.status_code == 200:
    # Index 0 means take the first dataframe from the extracted list
    df_djia = pd.read_html(res_djia.content)[0][cols]
    df_djia['Market Cap'] = df_djia['Market Cap'].apply(lambda x: x.replace(',', '').replace('B', '')).astype(float)
    df_djia = df_djia.sort_values(by='Market Cap', ascending=False)
    df_djia = df_djia.rename(columns={'Market Cap': 'Market Cap ($B)'})
    print(df_djia)

   Symbol                                 Company Name  Market Cap ($B)  \
0    AAPL                                    Apple Inc          3594.31   
1    MSFT                        Microsoft Corporation          3373.97   
2    AMZN                             Amazon.com, Inc.          2005.56   
3     JPM                         JPMorgan Chase & Co.           603.19   
4     WMT                                 Walmart Inc.           559.91   
5       V                                    Visa Inc.           537.17   
6     UNH              UnitedHealth Group Incorporated           474.34   
7      PG                 The Procter & Gamble Company           388.43   
8     JNJ                            Johnson & Johnson           359.17   
9      HD                         The Home Depot, Inc.           355.45   
10    MRK                            Merck & Co., Inc.           324.50   
11    CVX                          Chevron Corporation           291.26   
12     KO                